In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurement = Base.classes.measurement

station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.

most_recent_date = session.query(measurement.date).order_by(-measurement.date).first()
most_recent_date

('2017-01-01')

In [24]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
twelve_months = dt.datetime(2017, 1, 1) - dt.timedelta(weeks = 48)
last_12 = session.query(measurement.station, measurement.date, measurement.prcp, measurement.tobs).\
    filter(measurement.date > twelve_months).\
    order_by(measurement.date).all()
print(last_12)
# Calculate the date one year from the last date in data set.
last_date = session.query(measurement.date).order_by(measurement.date).first()
print(last_date)

one_year =  dt.datetime(2010, 1, 1) + dt.timedelta(weeks=52)

print(one_year)

# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




[('USC00519397', '2016-02-01', 0.0, 69.0), ('USC00513117', '2016-02-01', 0.0, 78.0), ('USC00514830', '2016-02-01', None, 77.0), ('USC00517948', '2016-02-01', None, 72.0), ('USC00519523', '2016-02-01', 0.0, 69.0), ('USC00519281', '2016-02-01', 0.07, 69.0), ('USC00516128', '2016-02-01', 0.15, 65.0), ('USC00519397', '2016-02-02', 0.06, 68.0), ('USC00513117', '2016-02-02', 0.41, 68.0), ('USC00514830', '2016-02-02', 0.2, 71.0), ('USC00517948', '2016-02-02', None, 69.0), ('USC00519523', '2016-02-02', 0.39, 70.0), ('USC00519281', '2016-02-02', 0.42, 68.0), ('USC00516128', '2016-02-02', 0.33, 69.0), ('USC00519397', '2016-02-03', 0.0, 70.0), ('USC00513117', '2016-02-03', 0.25, 71.0), ('USC00517948', '2016-02-03', None, 70.0), ('USC00519523', '2016-02-03', 0.03, 73.0), ('USC00519281', '2016-02-03', 0.2, 69.0), ('USC00516128', '2016-02-03', 0.26, 69.0), ('USC00519397', '2016-02-04', 0.0, 66.0), ('USC00513117', '2016-02-04', 0.0, 70.0), ('USC00514830', '2016-02-04', 0.0, 76.0), ('USC00517948', '20

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()